In [63]:
author="Jumabek Alikhanov"
date = 'Nov 22,2019'

This is Final Project material for the "Deep Learning" class I took  

## Download the dataset from https://www.unb.ca/cic/datasets/ids-2017.html
We only need CSV files that is preprocessed and labeled for ML 

In [64]:
#imports
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
# load data
dataroot = 'MachineLearningCVE/'
SEED=2

In [69]:
label_names=  ['BENIGN', 'PortScan', 'Bot', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed', 'DDoS',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'Infiltration', 'FTP-Patator',
       'SSH-Patator']

In [70]:
%autoreload 2
from preprocessing import load_data
X,y = load_data(dataroot) # reads csv file and returns np array of X,y -> of shape (N,D) and (N,1)

MachineLearningCVE/*.pcap_ISCX.csv
there are 2830743 flow records with 79 feature dimension
stripped column names
dropped bad columns
There are 0 nan entries
converted to numeric


In [71]:
from preprocessing import balance_data, normalize
X = normalize(X)

In [118]:

%autoreload 2
from models import Classifier

def ensure_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def getClassifier(args,runs_dir=None):
    
    (method,optim,lr,reg,batch_size,input_dim,num_class,num_epochs) = args
    if runs_dir is not None:
        ensure_dir(runs_dir)
    
    clf = Classifier(method,input_dim,num_class,lr=lr,reg=reg,num_epochs=num_epochs,
                        batch_size=batch_size,runs_dir=runs_dir)
    return clf


In [93]:
sample_indices = [ 475690, 2654438, 1835983, 1207146,  202083] # specifically chosen to contain non benign classes

In [116]:
#demo_y = [ 1,  0,  0,  2, 10]
demo_y = y[sample_indices]
demo_X = X[sample_indices,:]

In [123]:
demo_X[0,:]

array([ 1.29969907e-04, -1.14628173e-01, -2.89461932e-05, -2.53278995e-05,
       -4.25815706e-05, -2.46312738e-05, -8.36422108e-03, -8.04888271e-03,
       -9.79689229e-03, -9.67078842e-03, -4.42831255e-02, -1.21027576e-02,
       -5.17109297e-02, -4.09200229e-02, -9.10326652e-03, -3.11070103e-02,
       -7.22257718e-02, -1.34871318e-03, -1.12110160e-01, -1.74610410e-02,
       -3.85976844e-02, -7.10584968e-02, -4.23392933e-03, -7.38678798e-02,
       -1.07578710e-02, -1.75843649e-02, -3.47396992e-02, -3.77924112e-03,
       -4.64464612e-02,  0.00000000e+00, -1.11278205e-04,  0.00000000e+00,
        8.09805215e-07,  2.16128137e-06, -2.12874729e-02, -3.49614071e-03,
       -1.13497991e-02, -3.80500555e-02, -5.07538840e-02, -6.16648234e-02,
       -2.17028782e-02, -3.53797562e-02, -4.64464612e-02, -2.42339200e-04,
        7.01929450e-01, -3.15844297e-01, -9.48231593e-02, -1.11278205e-04,
       -2.43399001e-04,  2.02884362e-03, -4.85403128e-02, -9.79689229e-03,
       -5.17109297e-02,  

In [124]:

#hyper-params
batch_size = 5*1024 # increasing batch size with more gpu added
optim = 'Adam'

input_dim = 76
num_class = len(np.unique(labels))
num_epochs = 60
lr = 1e-3
reg = 1e-6


accuracies = {}
best_model = None
best_acc = -1
architecture = '1D-CNN_conv_2_fc_1'
run_number = 4
method='cnn2'



classifier_args = (method,optim,lr,reg,batch_size,input_dim,num_class,num_epochs)
config =  '{}/{}th_run/optim_{}_lr_{}_reg_{}_bs_{}'.format(architecture,run_number,optim,lr,reg,batch_size)
runs_dir = join(dataroot,'runs',config)
#loads classifier with the given parameter, threows error if classifier checkpoint is not found
clf = getClassifier(classifier_args,runs_dir)        
        
pred = clf.predict(demo_X,eval_mode=True)
for i in range(pred.shape[0]):
    print('True Class:{:20} --> Predicted Class: {:20} '.format(label_names[demo_y[i]],label_names[pred[i]]))

Loaded MachineLearningCVE/runs/1D-CNN_conv_2_fc_1/4th_run/optim_Adam_lr_0.001_reg_1e-06_bs_5120 model trained with batch_size = 5120, seen 3 epochs and 352 mini batches
True Class:PortScan             --> Predicted Class: PortScan             
True Class:BENIGN               --> Predicted Class: BENIGN               
True Class:BENIGN               --> Predicted Class: BENIGN               
True Class:Bot                  --> Predicted Class: Bot                  
True Class:Web Attack � XSS     --> Predicted Class: Web Attack � XSS     


In [126]:
print("Demo Prediction score",np.mean(pred==demo_y)*100 )

Demo Prediction score 100.0
